# 2. Grouping customers together!

#### Data Cleaning

We need to pre_process our data to delete all the outliers, NA or mistakes and set the time variables in a right form.

In [ ]:
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import preprocessing
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd
import numpy as np
import pyspark
from pyspark.sql import SparkSession
import findspark
import sklearn.metrics
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [2]:
bank_data = pd.read_csv("bank_transactions.csv")
bank_data.dropna(inplace=True)
bank_data.CustomerDOB = pd.to_datetime(bank_data.CustomerDOB)
bank_data.TransactionDate = pd.to_datetime(bank_data.TransactionDate)
bank_data.TransactionTime = bank_data.TransactionTime.apply(lambda x: datetime.strptime(str(x).zfill(6), "%H%M%S"))
bank_data = bank_data.rename(columns={'TransactionAmount (INR)':'TransactionAmount'})
bank_data["TransactionTime"] = bank_data["TransactionTime"].dt.hour

In [ ]:
bank_data.CustomerDOB.dt.year.hist(bins=50)

Watching the histogram we can see that there are a lot of people born in 1800 and others in 2050 and after. The last range is a mistake for the millenium bug, so we can fix it by subtracting 100 years in each of this data.

In [ ]:
bank_data.drop(bank_data[bank_data.CustomerDOB.dt.year == 1800].index, axis=0, inplace=True)
bank_data.loc[bank_data.CustomerDOB.dt.year > 2000, 'CustomerDOB'] = bank_data.loc[bank_data.CustomerDOB.dt.year > 2000, 'CustomerDOB'] - pd.DateOffset(years = 100)

In [ ]:
bank_data.CustomerDOB.dt.year.hist(bins=50)

Now the histogram shows right values of birthdays.

In [ ]:
bank_data.to_pickle("bankdata")
data = pd.read_pickle("bankdata")

## 2.1 Getting your data + feature engineering

#### Define a new index
Now our purpose is to group by the transaction for each costumer and add new variables to create a dataset of 27 features.
Since we discovered that the CustomerID field is not a good approximation of the customer identity, a combination of fields is being used to identify the user. We will perform the operation of grouping by using newly defined columns which we believe may represent a single customer. In the following cells two strategies are explored, in the first one the location in which a single customer does the transaction is assumed to be not changing, in the second one the balance of a single customer is considered consant. The combination of one of these two fields, along with DOB and gender will be the new identification column.

In [ ]:
#New columns
data["NewID1"] = data["CustomerDOB"].astype(str) + data["CustGender"] + data["CustLocation"]
data["NewID2"] = data["CustomerDOB"].astype(str) + data["CustGender"] + data["CustAccountBalance"].astype(str)

In [ ]:
data[["CustomerID", "NewID1", "CustAccountBalance", "TransactionAmount"]].groupby(["NewID1"]).agg(lambda x: x.tolist())

In [ ]:
data[["CustomerID", "NewID2", "CustLocation", "TransactionAmount"]].groupby(["NewID2"]).agg(lambda x: x.tolist())

#### Observations
As we can see, we get a list of equal balances when we cosider the location constant and equal locations when considering the balance constant, also worth noting that the CustomerIDs are indeed different. Noting finally that the size of the two grouped by datasets is similar, we can conclude that these two operations yield similar results, corroborating the idea that these new definitions for the customer identity are more refined than CustomerID. One final note is that the majority of customers has multiple associated CustomerIDs. We now proceed to making the dataset, the new index will be the combination of DOB, gender and balance, as it yields slightly more rows.

In [ ]:
data['Trans_over_100'] =  data.loc[data['TransactionAmount']>100,'TransactionAmount']
data['Small_transaction'] = data.loc[data['TransactionAmount']<100,'TransactionAmount']
data['Perc_transaction_balance'] = (data['TransactionAmount']/(data['CustAccountBalance']))*100
data['Big_balance'] = data.loc[data['CustAccountBalance']>100000,'CustAccountBalance']
data['Utilisation'] = data['CustAccountBalance']-data['TransactionAmount']
data['CustomerAge'] = ((pd.to_datetime('today') - data.CustomerDOB) / np.timedelta64(1,'Y')).astype(int)
data['Day_week'] = data.TransactionDate.dt.day_of_week
data['Month'] = data.TransactionDate.dt.month
data['Summer'] = data.loc[(data.TransactionDate.dt.month>=6) & (data.TransactionDate.dt.month<=9),'TransactionAmount']
data['Winter'] = data.loc[(data.TransactionDate.dt.month>=12) | (data.TransactionDate.dt.month<=2),'TransactionAmount']
data['Begin_month'] = data.loc[data.TransactionDate.dt.day<=15,'TransactionAmount']

In [ ]:
dataframe={
           'Transactions_#':list(data.groupby("NewID2").TransactionID.count()), # Number of transactions
           'Ts_over_100':list(data.groupby("NewID2").Trans_over_100.count()), # Number of transactions over 100 units of currency
           'Small_transactions_#':list(data.groupby("NewID2").Small_transaction.count()), # Number of transactions under 100 units of currency
           'Ave_transaction':list(data.groupby("NewID2").TransactionAmount.mean()), # The average transaction
           'Min_transaction':list(data.groupby("NewID2").TransactionAmount.min()), # The minimum transaction
           'Max_transaction':list(data.groupby("NewID2").TransactionAmount.max()), # The maximum transaction
           'Balance':list(data.groupby("NewID2").CustAccountBalance.first()), # The average balance
           'Mean_%_Tr_balance':list(round((data.groupby("NewID2").Perc_transaction_balance.mean()),1)), # The percentage of the transactions on the balance
           'Average_util':list(data.groupby("NewID2").Utilisation.mean()), # The average utilisation
           'Gender':list(data.groupby("NewID2").CustGender.first()), # The gender most common
           'Location':list(data.groupby("NewID2").CustLocation.agg(lambda x: pd.Series.mode(x)[0])), # The location most common
           'Age':list(data.groupby("NewID2").CustomerAge.first().astype(int)), # The average of the ages
           'Big_balance':list(data.groupby("NewID2").Big_balance.count()), # A boolean for people under (0) or over (1) 100000 in balance
           'Day_week':list(data.groupby("NewID2").Day_week.agg(lambda x: pd.Series.mode(x)[0])), # The most common day of the week
           'Month':list(data.groupby("NewID2").Month.agg(lambda x: pd.Series.mode(x)[0])), # The most common month
           'Summer':list(data.groupby("NewID2").Summer.sum()), # The amount of transactions during summer
           'Winter':list(data.groupby("NewID2").Winter.sum()), # The amount of transactions during winter
           'Begin_month':list(data.groupby("NewID2").Begin_month.sum()), # The amount of transactions during the first half of all months
           "Accounts_#":list(data.groupby("NewID2").CustomerID.count()), # The number of accounts that have each customer
           "Total_expenses":list(data.groupby("NewID2").TransactionAmount.sum()), # The sum of all transactions of a customer
           "1st_quart_Tr":list(data.groupby("NewID2").TransactionAmount.quantile(0.25)), # First quartile of the transactions amount
           "3rd_quart_Tr":list(data.groupby("NewID2").TransactionAmount.quantile(0.75)), # Third quartile of the transactions amount
           "Median_Tr":list(data.groupby("NewID2").TransactionAmount.median()), # Median of the transactions amount
           "Hour":list(data.groupby("NewID2").TransactionTime.agg(lambda x: pd.Series.mode(x)[0])), # Most common hour of the day
           "Median_util":list(data.groupby("NewID2").Utilisation.median()), # Median of utilisation
           "1st_quart_util":list(data.groupby("NewID2").Utilisation.quantile(0.25)), # First quartile of utilisation
           "3rd_quart_util":list(data.groupby("NewID2").Utilisation.quantile(0.75)) # Third quartile of utilisation
           
          }

In [ ]:
df=pd.DataFrame(dataframe)
df.to_pickle("2.1")

In [ ]:
df = pd.read_pickle("2.1")

## 2.2 Choose your features (variables)!

In order to do the Principal Component Analysis we want to take only numerical variables and normalize them. This is done for a better comparison between variables that have completely different values or unit of measure.

In [ ]:
features = ['Transactions_#','Ts_over_100','Small_transactions_#','Ave_transaction','Min_transaction','Max_transaction','Balance','Average_util','Age','Big_balance','Day_week','Month','Summer','Begin_month',"Accounts_#","Total_expenses","1st_quart_Tr","3rd_quart_Tr","Median_Tr","Median_util","1st_quart_util","3rd_quart_util"]

x = df.loc[:, features].values

# Standardizing and normalizing the features
x = StandardScaler().fit_transform(x)
z = normalize(x)
newdf=pd.DataFrame(z, columns=features)

Now that we have our dataset rescaled and normalized, we can use the method of the Principal Component Analysis.

In [ ]:
# Instantiate PCA
pca = PCA()
# Determine transformed features
z = pca.fit_transform(z)
# Determine explained variances and the cumulative sum
exp_var_pca = pca.explained_variance_ratio_
cum_sum_eigenvalues = np.cumsum(exp_var_pca)
# Plot
plt.bar(range(0,len(exp_var_pca)), exp_var_pca, alpha=0.5, align='center', label='Individual explained variance')
plt.step(range(0,len(cum_sum_eigenvalues)), cum_sum_eigenvalues, where='mid',label='Cumulative explained variance')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal components')
plt.legend(loc='best')
plt.tight_layout()
plt.show()

In [ ]:
len(cum_sum_eigenvalues[cum_sum_eigenvalues<0.70])+1

We can notice that the first 4 principal components explain more than 70% of variance so now we want to create a dataframe with only these 4 components.

In [ ]:
pca1 = PCA(n_components=4)
PCA_f = pca1.fit_transform(newdf)
PCA_DF = pd.DataFrame(data = PCA_f, columns = ['pc1', 'pc2','pc3', 'pc4'])
PCA_DF.to_pickle("pcadf")

## 2.3 Clustering!

#### MapReduce in pySpark

In [ ]:
df = pd.read_pickle("pcadf")

In [ ]:
# Pyspark initialization
findspark.init()
spark = SparkSession.builder.master("local[4]").getOrCreate()

In [ ]:
df = pd.read_pickle("pcadf")
values = df.values.tolist()
sdf = spark.createDataFrame(values,["pc1","pc2","pc3","pc4"])

In the following cell is presented the code for the kmeans algorithm

In [ ]:
k = 5
rdd = sdf.rdd.map(list)
vectorized_rdd = rdd.map(lambda row: np.array(row))

def distance(x, y):
    distance = np.zeros(len(y))
    for i in range(len(y)):         
        distance[i] = np.linalg.norm(x-y[i])
    key = np.argmin(distance)
    return (key, x)

init_c = np.array(df.sample(k))

diff = 999
while diff > 0.01:  # threshold for shortening computation time
    d = vectorized_rdd.map(lambda row: distance(row,init_c))
    updated_c = d.groupByKey().mapValues(lambda row: sum(row)/len(row)).collect()
    
    K_new = {}                               #
    for i in updated_c:                      # 
        K_new[i[0]]=i[1]                     # this block of code exists to keep
    finalist = []                            # track of the index of each centroid
    for i in range(k):                       #
        finalist.append(np.array(K_new[i]))  #
    final_c = np.array(finalist)
    
    diff = np.linalg.norm(final_c-init_c)
    init_c = final_c
    print(diff)  # printing out the difference value to monitor the execution of the algorithm
    
print(init_c)

#### Optimal clustering
We will use the methods of the elbow and the silhouette to find out what is the optimal number of clusters in terms of scoring function and overlap. In this section will be used the already implemented version of kmeans inside sklearn

In [ ]:
model = {}
for k in range(1,10):
    kmeans = KMeans(n_clusters=k).fit(df)
    model[k] = kmeans.inertia_
    
plt.figure()
plt.plot(list(model.keys()), list(model.values()))
plt.xlabel('Clusters')
plt.ylabel('Cost')
plt.title('Elbow')
plt.show()

The elbow method gives as optimal numbers of clusters a range between 4 and 6

Silhouette scores:

In [ ]:
for i in range(2,10):
    labels = KMeans(n_clusters=i).fit(df).labels_
    print(str(i)+"=="+str(sklearn.metrics.silhouette_score(df,labels,sample_size=1000)))

#### Observations
As we can see from the values of the silhouette scores, the clusters could have a bit of overlap. Nevertheless we can conclude that in this case the optimal number of clusters will be 5 as it yields the highest silhouette score and it is in line with the results of the elbow method.

Comparing our kmeans algorithm with the version of kmeans++ implemented in the sklearn package, it can be seen that the resulting centroids are very similar vecors (the order is shuffled), this discrepancy is due to the threshold we opted to use in our version for time saving reasons. This last consideration points out the main difference between the sklearn version and ours, which is the running time, as the former is way faster.

In [ ]:
kmeans = KMeans(n_clusters=5, init='k-means++').fit(df)
kmeans.cluster_centers_

In [ ]:
final_c

## 2.4 Analysing your results!

We created a new column that shows the cluster to which each row belongs. Then we choose the average transaction, balance and total expences as more relevant variables relevant to identify the cluster of each customer. With the three functions below we want to divide in 4 classes each variable.

In [ ]:
kmeans = KMeans(n_clusters=5, init='k-means++')
df["cluster"] = kmeans.fit_predict(df)

In [ ]:
ogdf = pd.read_pickle("2.1")

In [ ]:
def classTransactions(x):
    if x < 1000: return("0-1000")
    elif x <2500: return("1000-2500")
    elif x < 4000: return("2500-4000")
    else: return(">4000")
        
def classBalance(x):
    if x < 10000: return("0-10000")
    elif x <50000: return("10000-50000")
    elif x < 200000: return("50000-200000")
    else: return(">200000")
    
def classExpenses(x):
    if x < 1000: return("0-1000")
    elif x <5000: return("1000-5000")
    elif x < 20000: return("5000-20000")
    else: return(">20000")

In [ ]:
ogdf["cluster"] = df["cluster"]
ogdf["Transactions_class"] = ogdf.Ave_transaction.apply(lambda row: classTransactions(row))
ogdf["Balance_class"] = ogdf.Balance.apply(lambda row: classBalance(row))
ogdf["Expenses_class"] = ogdf.Total_expenses.apply(lambda row: classExpenses(row))

Now that we have 3 new columns to show the class to which each row belongs, we want to create for each variable a pivot table normalized for rows.

In [ ]:
pivot1 = round(pd.crosstab(index=ogdf.cluster, columns=ogdf.Transactions_class, values=ogdf.Ave_transaction, aggfunc="count", normalize="index")*100,1)
pivot2 = round(pd.crosstab(index=ogdf.cluster, columns=ogdf.Balance_class, values=ogdf.Balance, aggfunc="count", normalize="index")*100,1)
pivot3 = round(pd.crosstab(index=ogdf.cluster, columns=ogdf.Expenses_class, values=ogdf.Total_expenses, aggfunc="count", normalize="index")*100,1)

In [ ]:
pivot1

In [ ]:
pivot2

In [ ]:
pivot3

#### Observations
1. Transactions: Clusters from 1 to 4 have a similar trend regarding the average transaction amount, in particular we see that the majority of the transactions belong to the lower ranges, and cluster 3 has a more extreme behaviour with respect to the other three.

2. Balance: Also in this case the first cluster has a different trend from the others, having all four classes with similar percentages. In cluster 1 and even more in clusters 2, 3 and 4 the percentage about the lower class increases a lot, reaching values above 50%. Going through the other classes we can notice that clusters 2, 3 and 4 have a similar trend with low values for balances above 200000.

3. Expenses: In the case of the total expenses, we can see that clusters 3 and 4 have a very similar trend of percentages across the classes. Cluster 1 has peaks in the higher classes, having also a very low percentage of small expenses. Cluster 0 exhibits a more balanced trend across the classes, hinting to a gaussian distribution.

Summing up these observations we can conclude that Cluster 0 is composed by the wealthier people, Cluster 1 contains people more prone to spend money. Clusters 2, 3 and 4 are more representative of the behaviour of an average consumer, with Cluster 2 deviating a bit towards the behaviour of the upper clusters.

Silhouette score evaluation:

In [ ]:
labels = KMeans(n_clusters=5).fit(df).labels_
print(str(sklearn.metrics.silhouette_score(df,labels)))

This silhouette score indicates a bit of overlap between the clusters, as expected after seeing the behaviours of the pivot tables and from the rough evaluation in point 2.3. The clustering performance in our opinion is overall acceptable.

## Algorithmic Question

The idea behind this algorithm is to tranform the time interval between opening and closure $t$ of an entrance into a segment of length $t$, doing so the maximum number of superpositions between these segments for each unit of time will be the minimum number of guards required.

In [ ]:
input_list = list(input().split())
M = list(map(int, input().split()))
doors = set(M)
array = np.array(M)

# transformation
D = {}
for i in doors:
    temp = np.where(array==i)[0]
    D[i] = set(range(min(temp),max(temp)+1))
    
# count of intersections
count = []
for i in range(len(array)):
    c=0
    for j in D.keys():
        if i in D[j]: c+=1
    count.append(c)

# output
if int(input_list[2])<max(count): print("NO")
else: print("YES")